In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
len(os.listdir('/kaggle/input/dogs-vs-cats/train/train'))

In [ ]:
#use VGG16 pretrained model
import tensorflow as tf

In [ ]:
vgg=tf.keras.applications.VGG16(include_top=False, input_shape=(224,224,3))

In [ ]:
vgg.summary()

In [ ]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
vgg.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
flatten_layer= Flatten()(vgg.output)
final_layer= Dense(2,activation='softmax')(flatten_layer)

In [ ]:
#assemble model
model= Model(inputs=vgg.input, outputs= final_layer)

In [ ]:
filenames= os.listdir('/kaggle/input/dogs-vs-cats/train/train')
labels=[]
for file in filenames:
    labels.append(file.split('.')[0])

In [ ]:
df= pd.DataFrame({'Filename': filenames, 'Class': labels})

In [ ]:
df.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen= ImageDataGenerator(zoom_range=0.2, shear_range=0.2, horizontal_flip= True, rescale= 1/255)

In [ ]:
train_data= data_gen.flow_from_dataframe(df,'/kaggle/input/dogs-vs-cats/train/train', 
                x_col='Filename', y_col='Class', target_size=(224,224), batch_size=32, class_mode= 'categorical' )

In [ ]:
train_data.class_indices

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

In [ ]:
model= Sequential()

In [ ]:
model.add(Conv2D(32,(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D())
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128,(5,5), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(2,activation='softmax'))



In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_data, steps_per_epoch=len(train_data), epochs=10)

In [ ]:
model.save('myModel1.h5')

In [ ]:
import tensorflow as tf
new_model= tf.keras.models.load_model('../input/traineddata/myModel1.h5')

In [ ]:
import cv2

In [ ]:
img0= cv2.imread('../input/dogs-vs-cats/train/train/cat.1001.jpg')

In [ ]:
img0.shape

In [ ]:
img0= cv2.resize(img0, (224,224))       #resize

In [ ]:
img0.shape

In [ ]:
img0=img0/255            #Scaling

In [ ]:
class_code= model.predict(img0.reshape(1,224,224,3)).argmax()

In [ ]:
'CAT' if class_code==0 else 'DOG'

In [ ]:
# Using Function
def get_cat_or_dog(filepath):
    img0=cv2.imread(filepath)
    img0=cv2.resize(img0,(224,224))/255
    class_code=model.predict(img0.reshape(1,224,224,3)).argmax()
    return 'CAT' if class_code==0 else 'DOG'

In [ ]:
get_cat_or_dog('../input/dogs-vs-cats/test/test/1.jpg')

In [ ]:
get_cat_or_dog('../input/dogs-vs-cats/test/test/10001.jpg')

In [ ]:
get_cat_or_dog('../input/myimages/IMG20200516131239.jpg')

In [ ]:
get_cat_or_dog('../input/myimages/IMG20200516134927.jpg')

In [ ]:
get_cat_or_dog('../input/myimages/IMG20210221120514.jpg')

In [ ]:
get_cat_or_dog('../input/myimages/IMG20210221120519.jpg')

In [ ]:
get_cat_or_dog('../input/myimages/IMG20210221120700.jpg')